In [4]:
import websocket
import json
from pprint import pprint
import pandas as pd
import requests
from apscheduler.schedulers.blocking import BlockingScheduler
import time

URL = "wss://ws-feed.pro.coinbase.com"

#products = requests.get('https://api.pro.coinbase.com/products/').json()
#pairs = [entry['id'] for entry in products]

pairs = ['BTC-USD', 'LTC-USD', '']

trades = {pair: pd.DataFrame() for pair in pairs}
ohlcvs = {pair: pd.DataFrame() for pair in pairs}

scheduler = BlockingScheduler()

def job(pair):
    
    global ohlcvs
    global trades
    
    df = trades[pair]
    
    df = df[['time', 'price', 'size', 'trade_id']]
    new_index = pd.DatetimeIndex(df['time'])
    df['size'] = df['size'].apply(lambda x: float(x))
    df = df.set_index(new_index).drop(columns='time')
    vol = df['size'].resample('1Min').sum()
    df['price'] = df['price'].astype(float)
    ohlc1m = df['price'].resample('1Min').ohlc()
    ohlcvs[pair] = pd.concat([ohlc1m, vol], axis=1)
    
def on_message(ws, message):
    """Callback executed when a message comes.
    Positional argument:
    message -- The message itself (string)
    """
    msg = json.loads(message)
    pprint(msg)
    
    global trades
    
    for pair in trades:
        if msg['product_id'] == pair:
            trades[pair] = trades[pair].append(msg, ignore_index=True)
            
        try:
            scheduler.add_job(job(pair), 'interval', minutes=1)
            scheduler.start()
        except:
            pass
          
def on_open(socket):
    """Callback executed at socket opening.
    Keyword argument:
    socket -- The websocket itself
    """
    params = {
        "type": "subscribe",
        "channels": [{"name": "matches", "product_ids": pairs}]
    }
    socket.send(json.dumps(params))
    
def main():
    """Main function."""
    ws = websocket.WebSocketApp(URL, on_open=on_open, on_message=on_message)
    ws.run_forever()
    
if __name__ == '__main__':
    main()

{'maker_order_id': '4e0593ae-4e9a-4ee6-a3ec-a44b2deba601',
 'price': '8124.62',
 'product_id': 'BTC-USD',
 'sequence': 11015815268,
 'side': 'sell',
 'size': '0.06033526',
 'taker_order_id': 'faf41e2f-0a9b-45ba-b0f4-dcb38a04ddc6',
 'time': '2019-10-15T21:35:48.805000Z',
 'trade_id': 76020652,
 'type': 'last_match'}


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
error from callback <function on_message at 0x11d23f378>: 'product_id'
error from callback <function on_message at 0x11d23f378>: 'product_id'


{'message': 'Failed to subscribe',
 'reason': 'LCT-USD is not a valid product',
 'type': 'error'}
{'channels': [{'name': 'matches', 'product_ids': ['BTC-USD', 'ETH-USD']}],
 'type': 'subscriptions'}
{'maker_order_id': 'fae86d04-963c-4c2f-a1f7-eb496dc23a98',
 'price': '178.13',
 'product_id': 'ETH-USD',
 'sequence': 7411608681,
 'side': 'sell',
 'size': '1.19832577',
 'taker_order_id': '99033219-5535-4f2f-bc3b-e16f5098b152',
 'time': '2019-10-15T21:35:49.379000Z',
 'trade_id': 51909468,
 'type': 'last_match'}
{'maker_order_id': '87cad116-3dd8-44b3-8180-53916e211a08',
 'price': '8122.68',
 'product_id': 'BTC-USD',
 'sequence': 11015815375,
 'side': 'buy',
 'size': '0.06187123',
 'taker_order_id': '2bbbd8d9-04af-43ae-ae22-a737c59df889',
 'time': '2019-10-15T21:35:50.317000Z',
 'trade_id': 76020653,
 'type': 'match'}
{'maker_order_id': '12afad09-ebbd-4a61-82da-00959a5a8544',
 'price': '8124.55',
 'product_id': 'BTC-USD',
 'sequence': 11015815425,
 'side': 'sell',
 'size': '0.12310980',
 't

{'maker_order_id': 'e132dae1-2dff-4c43-a991-020b430b0e81',
 'price': '8124.51',
 'product_id': 'BTC-USD',
 'sequence': 11015816799,
 'side': 'buy',
 'size': '0.13831600',
 'taker_order_id': 'd99b063b-6af2-4b07-baba-4ff20b454702',
 'time': '2019-10-15T21:36:11.154000Z',
 'trade_id': 76020676,
 'type': 'match'}
{'maker_order_id': '9adbda1a-aee6-4cff-a331-120f9d1dbc8c',
 'price': '8124.28',
 'product_id': 'BTC-USD',
 'sequence': 11015816801,
 'side': 'buy',
 'size': '0.00897994',
 'taker_order_id': 'd99b063b-6af2-4b07-baba-4ff20b454702',
 'time': '2019-10-15T21:36:11.154000Z',
 'trade_id': 76020677,
 'type': 'match'}
{'maker_order_id': '3db95325-182e-4235-b85e-9971ce166573',
 'price': '8123.67',
 'product_id': 'BTC-USD',
 'sequence': 11015816803,
 'side': 'buy',
 'size': '0.42117541',
 'taker_order_id': 'd99b063b-6af2-4b07-baba-4ff20b454702',
 'time': '2019-10-15T21:36:11.154000Z',
 'trade_id': 76020678,
 'type': 'match'}
{'maker_order_id': '114ac85a-e648-496a-8e39-9bf615c878e3',
 'price'

{'maker_order_id': '6f042ee1-5d9c-4d33-8dda-bbf2022230f8',
 'price': '8124.96',
 'product_id': 'BTC-USD',
 'sequence': 11015817843,
 'side': 'sell',
 'size': '0.01000000',
 'taker_order_id': 'fb2bc4ce-1a46-4eef-b73a-d5df6252332b',
 'time': '2019-10-15T21:36:24.069000Z',
 'trade_id': 76020696,
 'type': 'match'}
{'maker_order_id': 'ac4ba9ea-d29a-4f7c-bb72-715068b1d6b6',
 'price': '8124.99',
 'product_id': 'BTC-USD',
 'sequence': 11015817845,
 'side': 'sell',
 'size': '0.00783610',
 'taker_order_id': 'fb2bc4ce-1a46-4eef-b73a-d5df6252332b',
 'time': '2019-10-15T21:36:24.069000Z',
 'trade_id': 76020697,
 'type': 'match'}
{'maker_order_id': 'ac4ba9ea-d29a-4f7c-bb72-715068b1d6b6',
 'price': '8124.99',
 'product_id': 'BTC-USD',
 'sequence': 11015817876,
 'side': 'sell',
 'size': '0.00404257',
 'taker_order_id': 'c4cdaacf-1bde-4b3a-802c-9d3604a73a29',
 'time': '2019-10-15T21:36:25.634000Z',
 'trade_id': 76020698,
 'type': 'match'}
{'maker_order_id': '02fdde74-e2c2-4582-8c8e-8adc3ea5b5b3',
 'pri

{'maker_order_id': '1727fe09-a6f5-4238-9de9-df7683344000',
 'price': '8131.02',
 'product_id': 'BTC-USD',
 'sequence': 11015819575,
 'side': 'sell',
 'size': '0.00697654',
 'taker_order_id': 'a8928ece-57f8-4fa6-b6a0-2a28399d7617',
 'time': '2019-10-15T21:36:46.003000Z',
 'trade_id': 76020719,
 'type': 'match'}
{'maker_order_id': '1727fe09-a6f5-4238-9de9-df7683344000',
 'price': '8131.02',
 'product_id': 'BTC-USD',
 'sequence': 11015819626,
 'side': 'sell',
 'size': '0.04707101',
 'taker_order_id': '0ca0965a-50b7-4426-a100-13613502aa03',
 'time': '2019-10-15T21:36:46.533000Z',
 'trade_id': 76020720,
 'type': 'match'}
{'maker_order_id': 'ef23eb75-4a54-4b71-b03e-e7875686a3a1',
 'price': '8129.16',
 'product_id': 'BTC-USD',
 'sequence': 11015819836,
 'side': 'buy',
 'size': '0.36665940',
 'taker_order_id': 'ccb58435-6963-4436-873a-9ba715af809c',
 'time': '2019-10-15T21:36:48.764000Z',
 'trade_id': 76020721,
 'type': 'match'}
{'maker_order_id': '4717970e-e72f-4030-aaeb-5f755cb364bd',
 'pric

{'maker_order_id': '12674fd4-b894-4685-9ccd-b1a5995155fc',
 'price': '8131.94',
 'product_id': 'BTC-USD',
 'sequence': 11015821476,
 'side': 'sell',
 'size': '0.03014091',
 'taker_order_id': '44db6f46-0dbf-449d-b4ba-5ac61da9248e',
 'time': '2019-10-15T21:37:06.706000Z',
 'trade_id': 76020742,
 'type': 'match'}
{'maker_order_id': '4135d3f0-8b58-4220-b089-ed3fb80b3a06',
 'price': '8131.91',
 'product_id': 'BTC-USD',
 'sequence': 11015821545,
 'side': 'sell',
 'size': '0.02353337',
 'taker_order_id': '50087b64-5e6c-46f1-98c4-2e716c5a8cc5',
 'time': '2019-10-15T21:37:07.593000Z',
 'trade_id': 76020743,
 'type': 'match'}
{'maker_order_id': '341a6c11-8b55-4b75-93c5-4e287039f700',
 'price': '178.4',
 'product_id': 'ETH-USD',
 'sequence': 7411613539,
 'side': 'sell',
 'size': '0.79183674',
 'taker_order_id': '28d2b91c-a7b0-44a3-ada9-65163f583fcf',
 'time': '2019-10-15T21:37:10.694000Z',
 'trade_id': 51909490,
 'type': 'match'}
{'maker_order_id': 'f32d5ada-3d2e-4971-b6dd-188d2cf6f338',
 'price'

{'maker_order_id': '366c090f-4571-428c-9d85-be2ef1d167e0',
 'price': '8133.37',
 'product_id': 'BTC-USD',
 'sequence': 11015823962,
 'side': 'buy',
 'size': '0.23351928',
 'taker_order_id': '17eedc96-6b22-411e-955f-a9bbd4eb5c3e',
 'time': '2019-10-15T21:37:26.074000Z',
 'trade_id': 76020761,
 'type': 'match'}
{'maker_order_id': 'c6415bf4-64ad-42b7-abea-25d9b595b0fb',
 'price': '8133.36',
 'product_id': 'BTC-USD',
 'sequence': 11015823964,
 'side': 'buy',
 'size': '1.00000000',
 'taker_order_id': '17eedc96-6b22-411e-955f-a9bbd4eb5c3e',
 'time': '2019-10-15T21:37:26.074000Z',
 'trade_id': 76020762,
 'type': 'match'}
{'maker_order_id': '3f46f997-2cef-4132-9d53-eb957f3be42f',
 'price': '8132.25',
 'product_id': 'BTC-USD',
 'sequence': 11015823966,
 'side': 'buy',
 'size': '1.00000000',
 'taker_order_id': '17eedc96-6b22-411e-955f-a9bbd4eb5c3e',
 'time': '2019-10-15T21:37:26.074000Z',
 'trade_id': 76020763,
 'type': 'match'}
{'maker_order_id': '4e1e5611-7ae6-42f1-98bc-44779a14d7e5',
 'price'

{'maker_order_id': '1d4b92f0-bf83-4e01-bfa1-b98f0b82ca76',
 'price': '178.52',
 'product_id': 'ETH-USD',
 'sequence': 7411615639,
 'side': 'buy',
 'size': '3.86511955',
 'taker_order_id': '702abda9-8a3b-4669-a615-a5cdc2913581',
 'time': '2019-10-15T21:37:34.088000Z',
 'trade_id': 51909506,
 'type': 'match'}
{'maker_order_id': '0418e878-faa5-47b5-9a9b-59a197ba01ad',
 'price': '8133.38',
 'product_id': 'BTC-USD',
 'sequence': 11015824762,
 'side': 'sell',
 'size': '0.00800000',
 'taker_order_id': '51325242-4f89-4e24-ab52-f16efea91d26',
 'time': '2019-10-15T21:37:34.582000Z',
 'trade_id': 76020781,
 'type': 'match'}
{'maker_order_id': 'c7ee5c94-7e0a-4a80-b59a-4fd5d9e8bf5b',
 'price': '8134.97',
 'product_id': 'BTC-USD',
 'sequence': 11015825026,
 'side': 'sell',
 'size': '0.00900000',
 'taker_order_id': '2633cdcf-2127-49cb-81f8-34bab2e44f0f',
 'time': '2019-10-15T21:37:37.659000Z',
 'trade_id': 76020782,
 'type': 'match'}
{'maker_order_id': '3c691569-e7eb-4f9b-b11b-1a7c3286ade8',
 'price'